# Raw Data Prep

## Libraries

In [1]:
import numpy as np
import pandas as pd
import xlrd
import os

In [2]:
import sys
np.set_printoptions(threshold=np.inf)

## Mount Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#os.listdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
os.listdir(".")

In [ ]:
housePath = "/content/drive/My Drive/Colab Notebooks/house_prices.xlsx"

# Raw Jobs Data into dataframe


In [ ]:
#xl2 = pd.ExcelFile("/content/drive/My Drive/Colab Notebooks/boise_msa_jobs_2000_to_2018.xlsx")
xl2 = pd.ExcelFile("./Data/boise_msa_jobs_2000_to_2018.xlsx")
rawJobs = xl2.parse("raw")
rawJobs.to_pickle("rawJobs.pkl")

In [ ]:
rawJobs.group.unique()

### From rawJobs.pkl to metro_jobs from pickle

In [ ]:
# open the pickle and turn it into a dataframe
rawJobs = pd.read_pickle("rawJobs.pkl")
metroJobs = rawJobs.copy()

# Splitting Strings
metroJobs['Occ_Cat_Code'], metroJobs['Occ_Sub_Cat_Code'] = metroJobs['occ_code'].str.split('-',1).str


# fixing the ['group'] column
metroJobs['group'] = metroJobs['group'].replace(np.nan,'detailed', regex=True)
metroJobs['occ_titl'] = metroJobs['occ_titl'].replace('Industry Total', 'All Occupations', regex = True)
metroJobs.loc[metroJobs.occ_titl == 'All Occupations', 'group'] = 'total'




# Assigning New Column Names
metroJobs = metroJobs.rename(columns ={'occ_titl': 'Occ_Title', 'group':'Cat_Type',
                                        'tot_emp': 'Emp_Count', 'h_mean':'Hr_Wage_Mean',
                                        'a_mean':'An_Wage_Mean', 'h_median': 'Hr_Median',
                                        'a_median':'An_Median','year':'Year'})



metroJobs['Year'] = metroJobs['Year'].astype(str)
metroJobs['Year'] = metroJobs['Year'].astype('datetime64[Y]')



# Assigning New Data Types
float_list = ['Emp_Count','Hr_Wage_Mean','An_Wage_Mean','Hr_Median','An_Median']

# Cleaning Column Strings for Data Typing
metroJobs[float_list] = metroJobs[float_list].replace('\W','',regex=True)

# Assigning new Data Types
#         need to use floats becuase int cannot handle missing
metroJobs = metroJobs.astype({ 'Cat_Type' : 'category',
                               'Occ_Cat_Code':'category','Occ_Sub_Cat_Code': 'category'})

metroJobs[float_list] = metroJobs[float_list].apply(pd.to_numeric, errors = 'coerce')


# Dropped Columns
metroJobs = metroJobs.drop(['prim_state','area','area_name','occ_code',
                              'emp_prse','mean_prse','h_wpct10','h_wpct25',
                             'h_wpct75','h_wpct90','a_wpct10','a_wpct25',
                             'a_wpct75','a_wpct90'], axis = 1)

# Assigning New Order to the Columns
metroJobs = metroJobs[['Year','Occ_Cat_Code','Occ_Sub_Cat_Code','Occ_Title','Cat_Type',
                        'Emp_Count','Hr_Wage_Mean','An_Wage_Mean','Hr_Median','An_Median']]

metroJobs['Occ_Title'] = metroJobs['Occ_Title'].str.title()
metroJobs['Emp_Count'] = metroJobs['Emp_Count'].fillna(0)
metroJobs['An_Wage_Mean'] = metroJobs['An_Wage_Mean'].fillna(0)



metroJobs.to_pickle('metroJobs.pkl')
metroJobs.head(5)

# Clean jobs for empolyee count for job type using major job categories

In [3]:
metroJobs = pd.read_pickle('metroJobs.pkl')

# start filtering to make df with Occ_Title as cols and the value be Emp_Count for each year.

# 0000 is the sub_cat # for the head of Cat
jobsF = metroJobs.copy()
jobsF = jobsF.loc[jobsF['Cat_Type'] != 'detailed']


jobsF['Occ_Title'] = jobsF['Occ_Title'].replace('Services','Service', regex = True)
jobsF['Occ_Title'] = jobsF['Occ_Title'].replace('Practitioners','Practitioner', regex = True)
jobsF['Occ_Title'] = jobsF['Occ_Title'].replace(' Mathematical Science Occupations',' Mathematical Occupations', regex = True)


jobsClean = jobsF.copy()
jobsClean.to_pickle('jobsClean.pkl')
jobsClean.head()


,Year,Occ_Cat_Code,Occ_Sub_Cat_Code,Occ_Title,Cat_Type,Emp_Count,Hr_Wage_Mean,An_Wage_Mean,Hr_Median,An_Median
0,2000-01-01,11,0000,Management Occupations,major,15790.0,2950.0,61360.0,2668.0,55490.0
24,2000-01-01,13,0000,Business And Financial Operations Occupations,major,9560.0,2150.0,44710.0,1942.0,40390.0
45,2000-01-01,15,0000,Computer And Mathematical Occupations,major,6020.0,2577.0,53600.0,2377.0,49440.0
54,2000-01-01,17,0000,Architecture And Engineering Occupations,major,12860.0,2178.0,45300.0,2062.0,42900.0
73,2000-01-01,19,0000,"Life, Physical, And Social Science Occupations",major,2650.0,1714.0,35650.0,1540.0,32040.0


## Create employee count % change

In [26]:
df = pd.read_pickle('jobsClean.pkl')
df = df[['Year','Occ_Title','Emp_Count']]

df['Job_Emp_Change'] = df.groupby('Occ_Title')['Emp_Count'].pct_change()

df = pd.pivot(df, values = 'Job_Emp_Change', index = 'Year', columns = 'Occ_Title')
df = df.reset_index()

df.at[6,'Construction And Extraction Occupations'] = np.nan

df.to_pickle('jobChange.pkl')
df.head()

Occ_Title,Year,All Occupations,Architecture And Engineering Occupations,"Arts, Design, Entertainment, Sports, And Media Occupations",Building And Grounds Cleaning And Maintenance Occupations,Business And Financial Operations Occupations,Community And Social Service Occupations,Computer And Mathematical Occupations,Construction And Extraction Occupations,"Education, Training, And Library Occupations",...,"Installation, Maintenance, And Repair Occupations",Legal Occupations,"Life, Physical, And Social Science Occupations",Management Occupations,Office And Administrative Support Occupations,Personal Care And Service Occupations,Production Occupations,Protective Service Occupations,Sales And Related Occupations,Transportation And Material Moving Occupations
0,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001-01-01,NaN,-0.113530,-0.091575,0.005284,-0.055439,-0.017606,-0.058140,-0.002198,-0.115068,...,-0.013699,-0.005988,-0.215094,0.016466,-0.020440,-0.042403,-0.134979,-0.005195,-0.025046,-0.045546
2,2002-01-01,0.016749,0.042105,0.044355,0.044678,0.038760,0.071685,0.091711,-0.134361,NaN,...,0.022436,0.042169,0.067308,-0.015576,0.047352,0.103321,-0.054038,0.075718,0.042036,0.025965
3,2003-01-01,0.011204,-0.409933,-0.019305,-0.021384,-0.027719,0.093645,-0.090468,0.026293,NaN,...,0.083595,-0.023121,0.099099,-0.013924,0.013793,0.076923,0.112323,0.063107,0.099955,0.000000
4,2004-01-01,0.020145,0.042796,0.220472,0.023136,0.055921,0.027523,0.190053,-0.007438,0.180599,...,0.054002,0.100592,-0.061475,0.099487,0.028723,-0.027950,-0.104443,0.155251,-0.013447,-0.043092


In [ ]:
#jobsClean[jobsClean['Cat_Type'] == 'major'].count()
#jobsClean.shape
#metroJobs.Cat_Type.nunique()

## Jobs by Industry, employee count

In [27]:
jobsF1 = pd.read_pickle('jobsClean.pkl')
jobsF1 = jobsF.drop(['Occ_Cat_Code', 'Occ_Sub_Cat_Code', 'Cat_Type'], axis =1)
jobsF1 = jobsF1.iloc[:,0:3]

jobsP1 = pd.pivot(jobsF1, values = 'Emp_Count', index = 'Year', columns = 'Occ_Title')
jobsP1 = jobsP1.reset_index()

# Gives the wrong answer, 2018 should be 706140
jobsP1['Job_Manual_Count'] = jobsP1.iloc[:,2:].sum(axis = 1)


# set manually because the 2000 data did not have a sum of all of the occupations
jobsP1.at[0,'All Occupations'] = 229780

jobsP1.to_pickle('jobTypes.pkl')
jobsP1.head()

Occ_Title,Year,All Occupations,Architecture And Engineering Occupations,"Arts, Design, Entertainment, Sports, And Media Occupations",Building And Grounds Cleaning And Maintenance Occupations,Business And Financial Operations Occupations,Community And Social Service Occupations,Computer And Mathematical Occupations,Construction And Extraction Occupations,"Education, Training, And Library Occupations",...,Legal Occupations,"Life, Physical, And Social Science Occupations",Management Occupations,Office And Administrative Support Occupations,Personal Care And Service Occupations,Production Occupations,Protective Service Occupations,Sales And Related Occupations,Transportation And Material Moving Occupations,Job_Manual_Count
0,2000-01-01,229780.0,12860.0,2730.0,7570.0,9560.0,2840.0,6020.0,13650.0,10950.0,...,1670.0,2650.0,15790.0,38160.0,2830.0,19040.0,3850.0,21960.0,14930.0,229780.0
1,2001-01-01,222100.0,11400.0,2480.0,7610.0,9030.0,2790.0,5670.0,13620.0,9690.0,...,1660.0,2080.0,16050.0,37380.0,2710.0,16470.0,3830.0,21410.0,14250.0,222100.0
2,2002-01-01,225820.0,11880.0,2590.0,7950.0,9380.0,2990.0,6190.0,11790.0,NaN,...,1730.0,2220.0,15800.0,39150.0,2990.0,15580.0,4120.0,22310.0,14620.0,216720.0
3,2003-01-01,228350.0,7010.0,2540.0,7780.0,9120.0,3270.0,5630.0,12100.0,NaN,...,1690.0,2440.0,15580.0,39690.0,3220.0,17330.0,4380.0,24540.0,14620.0,215740.0
4,2004-01-01,232950.0,7310.0,3100.0,7960.0,9630.0,3360.0,6700.0,12010.0,11440.0,...,1860.0,2290.0,17130.0,40830.0,3130.0,15520.0,5060.0,24210.0,13990.0,232950.0


## Clean jobs for empolyee count for Salery Range not using major job categories

## Jobs by Salery Range, empolyee count

In [28]:
df = pd.read_pickle('metroJobs.pkl')

df = df.loc[df['Cat_Type'] == 'detailed']
df = df[['Year', 'Emp_Count' ,'An_Wage_Mean']]

jobsSalery = df.groupby([df.Year,pd.cut(df.An_Wage_Mean,[10000,40000,70000,100000,150000,200000,300000]).astype(str)])[['Emp_Count']].sum().reset_index()

jobsSalery = pd.pivot(jobsSalery, values = 'Emp_Count', index = 'Year', columns = 'An_Wage_Mean')
jobsSalery = jobsSalery.reset_index()
jobsSalery = jobsSalery.replace(np.nan,0)

# fix the order of the columns
jobsSalery = jobsSalery[['Year','(10000.0, 40000.0]','(40000.0, 70000.0]','(70000.0, 100000.0]',
          '(100000.0, 150000.0]', '(150000.0, 200000.0]', '(200000.0, 300000.0]' ]]

jobsSalery.to_pickle('jobSalery.pkl')
jobsSalery

An_Wage_Mean,Year,"(10000.0, 40000.0]","(40000.0, 70000.0]","(70000.0, 100000.0]","(100000.0, 150000.0]","(150000.0, 200000.0]","(200000.0, 300000.0]"
0,2000-01-01,150080.0,34980.0,3710.0,0.0,0.0,0.0
1,2001-01-01,135860.0,45570.0,4520.0,0.0,0.0,0.0
2,2002-01-01,129450.0,40330.0,5850.0,1100.0,0.0,0.0
3,2003-01-01,132290.0,40650.0,4960.0,1160.0,0.0,0.0
4,2004-01-01,143120.0,44460.0,5710.0,2030.0,0.0,0.0
5,2005-01-01,147750.0,53520.0,4010.0,3070.0,220.0,0.0
6,2006-01-01,159950.0,50600.0,9800.0,3970.0,200.0,0.0
7,2007-01-01,167110.0,60680.0,20230.0,3950.0,240.0,0.0
8,2008-01-01,162520.0,64660.0,22890.0,4680.0,250.0,0.0
9,2009-01-01,146070.0,65420.0,23110.0,1280.0,230.0,590.0


## Percent change in # of emp for wage range

In [34]:
df = pd.read_pickle('metroJobs.pkl')

df = df.loc[df['Cat_Type'] == 'detailed']
df = df[['Year', 'Emp_Count' ,'An_Wage_Mean']]

df = df.groupby([df.Year,pd.cut(df.An_Wage_Mean,[10000,40000,70000,100000,150000,200000,
                                                 300000]).astype(str)])[['Emp_Count']].sum().reset_index()

df['Salery_Count_Change'] = df.groupby('An_Wage_Mean')['Emp_Count'].pct_change()

df = pd.pivot(df, values = 'Salery_Count_Change', index = 'Year', columns = 'An_Wage_Mean')
df = df.reset_index()

# These are the first values for that range, nothing to compare against so NaN
df.at[5, '(150000.0, 200000.0]'] = np.nan
df.at[9, '(200000.0, 300000.0]'] = np.nan

# Not sure what to do when values go from 220 - 0 and 0 - 220, setting to NaN for now.
#df.at[16, '(150000.0, 200000.0]'] = np.nan
df.at[18, '(150000.0, 200000.0]'] = np.nan

df = df[['Year','(10000.0, 40000.0]','(40000.0, 70000.0]','(70000.0, 100000.0]',
          '(100000.0, 150000.0]', '(150000.0, 200000.0]', '(200000.0, 300000.0]' ]]

df.to_pickle('salChange.pkl')
df

An_Wage_Mean,Year,"(10000.0, 40000.0]","(40000.0, 70000.0]","(70000.0, 100000.0]","(100000.0, 150000.0]","(150000.0, 200000.0]","(200000.0, 300000.0]"
0,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1,2001-01-01,-0.094749,0.302744,0.218329,NaN,NaN,NaN
2,2002-01-01,-0.047181,-0.114988,0.294248,NaN,NaN,NaN
3,2003-01-01,0.021939,0.007935,-0.152137,0.054545,NaN,NaN
4,2004-01-01,0.081866,0.093727,0.151210,0.750000,NaN,NaN
5,2005-01-01,0.032350,0.203779,-0.297723,0.512315,NaN,NaN
6,2006-01-01,0.082572,-0.054559,1.443890,0.293160,-0.090909,NaN
7,2007-01-01,0.044764,0.199209,1.064286,-0.005038,0.200000,NaN
8,2008-01-01,-0.027467,0.065590,0.131488,0.184810,0.041667,NaN
9,2009-01-01,-0.101218,0.011754,0.009611,-0.726496,-0.080000,NaN


In [ ]:
job1 = pd.read_pickle('jobTypes.pkl')
job2 = pd.read_pickle('jobSalery.pkl')

jobFeatures = pd.merge(job1, job2, on = 'Year', how = 'left')
jobFeatures = jobFeatures.drop(['Job_Manual_Count'], axis = 1)

jobFeatures.to_pickle('jobFeatures.pkl')
jobFeatures.info()

In [5]:
df1 = pd.read_pickle('jobChange.pkl')
df2 = pd.read_pickle('salChange.pkl')

df = pd.merge(df1, df2, on = 'Year', how = 'left')

df.to_pickle('jobShift.pkl')
df


,Year,All Occupations,Architecture And Engineering Occupations,"Arts, Design, Entertainment, Sports, And Media Occupations",Building And Grounds Cleaning And Maintenance Occupations,Business And Financial Operations Occupations,Community And Social Service Occupations,Computer And Mathematical Occupations,Construction And Extraction Occupations,"Education, Training, And Library Occupations",...,Production Occupations,Protective Service Occupations,Sales And Related Occupations,Transportation And Material Moving Occupations,"(10000.0, 40000.0]","(40000.0, 70000.0]","(70000.0, 100000.0]","(100000.0, 150000.0]","(150000.0, 200000.0]","(200000.0, 300000.0]"
0,2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001-01-01,NaN,-0.113530,-0.091575,0.005284,-0.055439,-0.017606,-0.058140,-0.002198,-0.115068,...,-0.134979,-0.005195,-0.025046,-0.045546,-0.094749,0.302744,0.218329,NaN,NaN,NaN
2,2002-01-01,0.016749,0.042105,0.044355,0.044678,0.038760,0.071685,0.091711,-0.134361,NaN,...,-0.054038,0.075718,0.042036,0.025965,-0.047181,-0.114988,0.294248,NaN,NaN,NaN
3,2003-01-01,0.011204,-0.409933,-0.019305,-0.021384,-0.027719,0.093645,-0.090468,0.026293,NaN,...,0.112323,0.063107,0.099955,0.000000,0.021939,0.007935,-0.152137,0.054545,NaN,NaN
4,2004-01-01,0.020145,0.042796,0.220472,0.023136,0.055921,0.027523,0.190053,-0.007438,0.180599,...,-0.104443,0.155251,-0.013447,-0.043092,0.081866,0.093727,0.151210,0.750000,NaN,NaN
5,2005-01-01,0.091779,0.056088,-0.077419,-0.042714,0.070613,-0.062500,0.058209,-1.000000,0.041084,...,0.160438,-0.053360,0.030153,0.063617,0.032350,0.203779,-0.297723,0.512315,NaN,NaN
6,2006-01-01,0.056108,0.159326,0.157343,0.114173,0.129001,0.111111,-0.016925,NaN,-0.004198,...,0.004997,-0.096033,0.050521,0.105511,0.082572,-0.054559,1.443890,0.293160,-0.090909,NaN
7,2007-01-01,0.057930,-0.052514,0.338369,0.071849,-0.012887,0.188571,0.051650,0.020878,0.015177,...,0.110497,0.115473,0.146183,0.181155,0.044764,0.199209,1.064286,-0.005038,0.200000,NaN
8,2008-01-01,0.006757,0.021226,0.045147,-0.039560,-0.004352,0.317308,-0.055935,-0.195595,-0.000831,...,-0.029353,0.186335,0.045954,-0.042717,-0.027467,0.065590,0.131488,0.184810,0.041667,NaN
9,2009-01-01,-0.077146,-0.081986,0.071274,-0.045767,-0.066434,0.000000,0.095376,-0.191656,0.046550,...,-0.257304,-0.055846,-0.081821,-0.052688,-0.101218,0.011754,0.009611,-0.726496,-0.080000,NaN
